In [3]:
import pandas as pd
import os
from pathlib import Path

In [5]:
# 데이터 경로
data_dir = Path("/Users/parkjuyong/Desktop/4-1/CareerRoute/assets/tiobe")

# 결과 저장 경로
output_dir = Path("/Users/parkjuyong/Desktop/4-1/CareerRoute/ml/trend_tech_ml/data")
output_dir.mkdir(parents=True, exist_ok=True) # 파일 없으면 자동으로 생성해줌

# 최종 파일 경로
output_file = output_dir / "integrated_tiobe_data.csv"

In [6]:
# 매핑 언어 딕셔너리로 정의
language_mapping = {
    'python': 'python',
    'java': 'java',
    'c++': 'cpp',
    'c#': 'csharp',
    'c': 'c',
    'javascript': 'javascript',
    'sql': 'sql',
    'r': 'r',
    'visual-basic': 'visual_basic',
    'perl': 'perl'
}

all_languages = list(language_mapping.keys()) # key값들로 리스트에 저장

In [14]:
# 테스트로 하나씩 로드
def load_csv(filepath: Path) -> pd.DataFrame:
    df=pd.read_csv(filepath)
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m') # 일까지 포함하면 시계열의 간격이 불규칙해짐
    return df

In [15]:
# 테스트
test_df = load_csv(data_dir / "python.csv")
test_df.head()

,date,percent
0,2001-06,1.25
1,2001-07,1.13
2,2001-08,1.20
3,2001-09,1.17
4,2001-10,1.28


In [17]:
# 모든 언어 csv로드
all_language = {} # 빈 딕셔너리

# 모든 파일 돌아가면서 컬럼을 위의 언어 매핑 방식으로 다시 정의 (c++의 이름으로 저장된 파일의 언어는 cpp로 컬럼 정의)
for lang_file, lang_col in language_mapping.items():
    csv_path = data_dir / f"{lang_file}.csv"

    if not csv_path.exists():
        print(f"{csv_path.name} 의 파일은 없습니다. 스킵합니다.")
        continue

    df = load_csv(csv_path)
    # percent컬럼을 언어 이름으로 바꿈(나중에 시계열 데이터로 한번에 나타낼때 컬럼이 날짜와 언어가 됨)
    df = df.rename(columns={'percent': lang_col}) 

    all_language[lang_col] = df
    print(f"{lang_file} 로드 완료 ({len(df)} rows)")
    

python 로드 완료 (291 rows)
java 로드 완료 (291 rows)
c++ 로드 완료 (291 rows)
c# 로드 완료 (291 rows)
c 로드 완료 (291 rows)
javascript 로드 완료 (291 rows)
sql 로드 완료 (130 rows)
r 로드 완료 (222 rows)
visual-basic 로드 완료 (186 rows)
perl 로드 완료 (291 rows)


In [18]:
all_language.keys()

dict_keys(['python', 'java', 'cpp', 'csharp', 'c', 'javascript', 'sql', 'r', 'visual_basic', 'perl'])

In [ ]:
# 언어별 df를 하나의 df로 머지함
merge_df = None

for lang, df in all_language.items():
    if merge_df is None:
        merge_df = df
    else:
        merge_df = pd.merge # 특정 컬럼 기준으로 가로로 합침
            merge_df,
            df,
            on='date', # 날짜 기준 머지(기준 컬럼)
            how='outer' # 합집합 개념
        )

In [21]:
merge_df.head()

,date,python,java,cpp,csharp,c,javascript,sql,r,visual_basic,perl
0,2001-06,1.25,26.49,14.20,0.38,20.24,1.55,2.96,NaN,NaN,6.60
1,2001-07,1.13,25.03,16.11,0.43,20.77,1.72,2.77,NaN,NaN,6.78
2,2001-08,1.20,24.66,16.12,0.38,20.75,1.66,2.38,NaN,NaN,7.07
3,2001-09,1.17,24.82,15.85,0.39,20.77,1.63,2.36,NaN,NaN,7.31
4,2001-10,1.28,25.68,16.10,0.42,19.75,1.51,2.24,NaN,NaN,7.38


In [26]:
# 날짜 정렬
merge_df = merge_df.sort_values('date').reset_index(drop=True) # date기준 오름차순 정렬 후 인덱스 재정의
merge_df.head()

,date,python,java,cpp,csharp,c,javascript,sql,r,visual_basic,perl
0,2001-06,1.25,26.49,14.20,0.38,20.24,1.55,2.96,NaN,NaN,6.60
1,2001-07,1.13,25.03,16.11,0.43,20.77,1.72,2.77,NaN,NaN,6.78
2,2001-08,1.20,24.66,16.12,0.38,20.75,1.66,2.38,NaN,NaN,7.07
3,2001-09,1.17,24.82,15.85,0.39,20.77,1.63,2.36,NaN,NaN,7.31
4,2001-10,1.28,25.68,16.10,0.42,19.75,1.51,2.24,NaN,NaN,7.38


In [33]:
# 겹치는 기간으로 필터링
START_DATE = "2010-09"

filtered_df = merge_df[merge_df['date'] >= START_DATE]
filtered_df = filtered_df.dropna()

In [36]:
filtered_df.to_csv(output_file, index=False)

print("저장 완료")
print(f"Rows: {len(filtered_df)}")
print(f"Date range: {filtered_df['date'].iloc[0]} ~ {filtered_df['date'].iloc[-1]}")
print("Languages:", list(filtered_df.columns[1:]))

저장 완료
Rows: 1117
Date range: 2018-02 ~ 2025-12
Languages: ['python', 'java', 'cpp', 'csharp', 'c', 'javascript', 'sql', 'r', 'visual_basic', 'perl']
